# Capstone Database Engineer Course by Meta
# Final Project Assessment
## Database Creation, Reading, Updating, and Deletion (CRUD) using .csv file, MySQL, MySQL WorkBench, Python, Jupyter Notebook, GitHub, and Visual Studio Code





# Initialization: Import required libraries.

In [43]:
#Import MySQL Connector/Python
import mysql.connector as connector
#Import the Error class from MySQL Connector/Python
from mysql.connector import Error
#Import Panda, required to create a dataframe with the data to record in the database
import pandas as pd

# Task 0: Connect to the Database designed and created using MySQL WorkBench.

In [44]:
#Define database configurations as a Python dictionary object called dbconfig
dbconfig={"host":"localhost","user":"root", "password":""}

In [45]:
#Connect to MySQL
def connectToMySQL():
    try:
        connection = connector.connect(**dbconfig)
        print("Successfully connected to MySQL.")
        return connection
    except Error as er:
        print("Something went wrong.")
        print("Error Code:", er.errno)
        print("Error Message:", er.msg)
        
cnx=connectToMySQL()

Successfully connected to MySQL.


In [46]:
#Create a cursor
cursor = cnx.cursor()
print("Cursor successfully created.")

Cursor successfully created.


In [47]:
#Set the database for use
try:
    use_database1_sql = """USE little_lemon_org_database""" 
    cursor.execute(use_database1_sql) 
    print("Database little_lemon_org_database is ready for use.")
except Error as er:
    print("Something went wrong.")
    print("Error Code:", er.errno)
    print("Error Message:", er.msg)

Database little_lemon_org_database is ready for use.


# Task 1: Tranform the data given and extract the data for each Table

In [6]:
#Create a dataframe with the .csv files provided
file_location_name = 'tables_data\LittleLemon_allData.csv'
df = pd.read_csv(file_location_name, index_col=False, delimiter = ',', keep_default_na=False, na_values=['']) 
df.head(5)

,Row Number,Order ID,Order Date,Delivery Date,Customer ID,Customer Name,City,Country,Postal Code,Country Code,...,Sales,Quantity,Discount,Delivery Cost,Course Name,Cuisine Name,Starter Name,Desert Name,Drink,Sides
0,1,54-366-6861,15-Jun-2020,26-Mar-2020,72-055-7985,Laney Fadden,Daruoyan,China,993-0031,CN,...,187.5,2,20.00,60.51,Greek salad,Greek,Olives,Greek yoghurt,Athens White wine,Tapas
1,2,63-761-3686,25-Aug-2020,17-Jul-2020,65-353-0657,Giacopo Bramich,Ongjin,North Korea,216282,KP,...,352.5,1,15.00,96.75,Bean soup,Italian,Flatbread,Ice cream,Corfu Red Wine,Potato salad
2,3,65-351-6434,17-Aug-2021,24-Apr-2020,90-876-6799,Lia Bonar,Quince Mil,Peru,663246,PE,...,112.5,3,10.52,36.37,Pizza,Italian,Minestrone,Cheesecake,Italian Coffee,Bruschetta
3,4,36-917-2834,14-Aug-2021,13-Apr-2020,73-873-4827,Merrill Baudon,Susaki,Japan,987-0352,JP,...,330.0,3,11.23,5.49,Carbonara,Turkish,Tomato bread,Affogato,Roma Red wine,Focaccia
4,5,86-114-9232,20-Dec-2020,2-Feb-2021,80-927-5246,Tasia Fautly,Tobruk,Libya,351 01,LY,...,480.0,2,51.05,63.64,Kabasa,Greek,Falafel,Turkish yoghurt,Ankara White Wine,Meatballs


In [7]:
# Get data types of columns before conversion
df.dtypes

Row Number         int64
Order ID          object
Order Date        object
Delivery Date     object
Customer ID       object
Customer Name     object
City              object
Country           object
Postal Code       object
Country Code      object
Cost             float64
Sales            float64
Quantity           int64
Discount         float64
Delivery Cost    float64
Course Name       object
Cuisine Name      object
Starter Name      object
Desert Name       object
Drink             object
Sides             object
dtype: object

In [8]:
# Convert datatype of appropriate columns to date time
df["Order Date"]=df["Order Date"].apply(pd.to_datetime)
df["Delivery Date"]=df["Delivery Date"].apply(pd.to_datetime)
# Convert datatype of appropriate columns to string
df["Order ID"]=df["Order ID"].astype('string')
df["Customer ID"]=df["Customer ID"].astype('string')
df["Customer Name"]=df["Customer Name"].astype('string')
df["City"]=df["City"].astype('string')
df["Country"]=df["Country"].astype('string')
df["Postal Code"]=df["Postal Code"].astype('string')
df["Country Code"]=df["Country Code"].astype('string')
df["Course Name"]=df["Course Name"].astype('string')
df["Cuisine Name"]=df["Cuisine Name"].astype('string')
df["Starter Name"]=df["Starter Name"].astype('string')
df["Desert Name"]=df["Desert Name"].astype('string')
df["Drink"]=df["Drink"].astype('string')
df["Sides"]=df["Sides"].astype('string')
# Add 2 digits for the decimal part of floats
df["Cost"]=df["Cost"].round(decimals=2)
df["Sales"]=df["Sales"].round(2)
df["Discount"]=df["Discount"].round(2)
df["Delivery Cost"]=df["Delivery Cost"].round(2)

In [9]:
# Check for missing values
df.isna().sum()

Row Number       0
Order ID         0
Order Date       0
Delivery Date    0
Customer ID      0
Customer Name    0
City             0
Country          0
Postal Code      0
Country Code     0
Cost             0
Sales            0
Quantity         0
Discount         0
Delivery Cost    0
Course Name      0
Cuisine Name     0
Starter Name     0
Desert Name      0
Drink            0
Sides            0
dtype: int64

In [10]:
# Generate the column AddressID with data from existing columns and a digit  
df["AddressID"]=df["City"].str.replace(' ', '', regex=False)+"-"+df["Country"].str[:3]+"-"+df["Postal Code"].str.replace('-', '', regex=False)+"-"+str(1)
df["AddressID"]=df["AddressID"].str.upper()

# Create the corresponding dataframe for Table DeliveryAddress
df_DeliveryAddress = df.filter(items=["AddressID", "City","Country","Postal Code","Country Code"])
# Rename columns as in DeliveryAddress Table
df_DeliveryAddress = df_DeliveryAddress.rename(columns={"Row Number":"AddressID","Postal Code":"PostalCode","Country Code":"CountryCode"})

# Delete dupplicate row because AddressID is a Primary key
df_DeliveryAddress = df_DeliveryAddress.drop_duplicates()

# Display first rows of DeliveryAddress Table
df_DeliveryAddress.head(5)

,AddressID,City,Country,PostalCode,CountryCode
0,DARUOYAN-CHI-9930031-1,Daruoyan,China,993-0031,CN
1,ONGJIN-NOR-216282-1,Ongjin,North Korea,216282,KP
2,QUINCEMIL-PER-663246-1,Quince Mil,Peru,663246,PE
3,SUSAKI-JAP-9870352-1,Susaki,Japan,987-0352,JP
4,TOBRUK-LIB-351 01-1,Tobruk,Libya,351 01,LY


In [11]:
# Generate the column ProductID with data from existing columns and a digit  
df["ProductID"] = df["Course Name"].str[:3] + df["Cuisine Name"].str[:3] + df["Starter Name"].str[:3] + df["Desert Name"].str[:3] + df["Drink"].str[:3] + df["Sides"].str[:3] + str(1)

# Create the corresponding dataframe for Table Products
df_Produts = df.filter(items=["ProductID", "Course Name","Cuisine Name","Starter Name","Desert Name","Drink","Sides"])
# Rename columns as in Products Table
df_Produts = df_Produts.rename(columns={"Course Name":"CourseName","Cuisine Name":"CuisineName",
                                                        "Starter Name":"StarterName","Desert Name":"DesertName"})
# Delete dupplicate row because ProductID is a Primary key
df_Produts = df_Produts.drop_duplicates()

# Display first rows of Products Table
df_Produts.head(5)

,ProductID,CourseName,CuisineName,StarterName,DesertName,Drink,Sides
0,GreGreOliGreAthTap1,Greek salad,Greek,Olives,Greek yoghurt,Athens White wine,Tapas
1,BeaItaFlaIceCorPot1,Bean soup,Italian,Flatbread,Ice cream,Corfu Red Wine,Potato salad
2,PizItaMinCheItaBru1,Pizza,Italian,Minestrone,Cheesecake,Italian Coffee,Bruschetta
3,CarTurTomAffRomFoc1,Carbonara,Turkish,Tomato bread,Affogato,Roma Red wine,Focaccia
4,KabGreFalTurAnkMea1,Kabasa,Greek,Falafel,Turkish yoghurt,Ankara White Wine,Meatballs


In [12]:
# Extract 2 new columns FirstName and LastName from the column Name
df["FirstName"]=df["Customer Name"].str.split(" ", n=1).str.get(0)
df["LastName"]=df["Customer Name"].str.split(" ", n=1).str.get(1)

# Create the corresponding dataframe for Table Customers
df_Customers = df.filter(items=["Customer ID", "FirstName","LastName"])
# Rename columns as in Customers Table
df_Customers = df_Customers.rename(columns={"Customer ID":"CustomerID"})

# Delete dupplicate row because CustomerID is a Primary key
df_Customers=df_Customers.drop_duplicates()

# Display first rows of Customers Table
df_Customers.head(5)

,CustomerID,FirstName,LastName
0,72-055-7985,Laney,Fadden
1,65-353-0657,Giacopo,Bramich
2,90-876-6799,Lia,Bonar
3,73-873-4827,Merrill,Baudon
4,80-927-5246,Tasia,Fautly


In [13]:
# Create the corresponding dataframe for Table Orders
df_Orders = df.filter(items=["Order ID", "Order Date","Customer ID"])
# Rename columns as in Customers Table
df_Orders = df_Orders.rename(columns={"Order ID":"OrderID","Order Date":"OrderDate","Customer ID":"CustomerID"})

# Delete dupplicate row because OrderID is a Primary key
df_Orders=df_Orders.drop_duplicates()

# Display first rows of Orders Table
df_Orders.head(5)

,OrderID,OrderDate,CustomerID
0,54-366-6861,2020-06-15,72-055-7985
1,63-761-3686,2020-08-25,65-353-0657
2,65-351-6434,2021-08-17,90-876-6799
3,36-917-2834,2021-08-14,73-873-4827
4,86-114-9232,2020-12-20,80-927-5246


In [14]:
# Create the corresponding dataframe for Table OrderDetails
df_OrderDetails = df.filter(items=["Delivery Date","Delivery Cost", "Quantity", "Cost", "Discount", "AddressID", "Order ID", "ProductID"])
# Rename columns as in DeliveryInfo Table
df_OrderDetails = df_OrderDetails.rename(columns={"Delivery Date":"DeliveryDate","Delivery Cost":"DeliveryCost","Cost":"CostPrice","Order ID":"OrderID"})

# Display first rows of OrderDetails Table
df_OrderDetails.head(5)

,DeliveryDate,DeliveryCost,Quantity,CostPrice,Discount,AddressID,OrderID,ProductID
0,2020-03-26,60.51,2,125.0,20.00,DARUOYAN-CHI-9930031-1,54-366-6861,GreGreOliGreAthTap1
1,2020-07-17,96.75,1,235.0,15.00,ONGJIN-NOR-216282-1,63-761-3686,BeaItaFlaIceCorPot1
2,2020-04-24,36.37,3,75.0,10.52,QUINCEMIL-PER-663246-1,65-351-6434,PizItaMinCheItaBru1
3,2020-04-13,5.49,3,220.0,11.23,SUSAKI-JAP-9870352-1,36-917-2834,CarTurTomAffRomFoc1
4,2021-02-02,63.64,2,320.0,51.05,TOBRUK-LIB-351 01-1,86-114-9232,KabGreFalTurAnkMea1


# Task 2: Insert data into the Tables.

In [15]:
%%time

#Insert records into the Table DeliveryAddress
try:
    insert_table_DeliveryAddress_sql = """INSERT INTO little_lemon_org_database.DeliveryAddress VALUES (%s,%s,%s,%s,%s)""" 
    for i,row in df_DeliveryAddress.iterrows(): 
        cursor.execute(insert_table_DeliveryAddress_sql, tuple(row)) 
        cnx.commit()
    print(i+1, "Rows were successfully recorded in the Table DeliveryAddress.")
except Error as er:
    print("Something went wrong.")
    print("Error Code:", er.errno)
    print("Error Message:", er.msg)

21000 Rows were successfully recorded in the Table DeliveryAddress.
CPU times: total: 15.5 s
Wall time: 20min 9s


In [16]:
%%time

#Insert records into the Table Products
try:
    insert_table_Products_sql = """INSERT INTO little_lemon_org_database.Products VALUES (%s,%s,%s,%s,%s,%s,%s)""" 
    for i,row in df_Produts.iterrows(): 
        cursor.execute(insert_table_Products_sql, tuple(row)) 
        cnx.commit()
    print(i+1, "Rows were successfully recorded in the Table Products.")
except Error as er:
    print("Something went wrong.")
    print("Error Code:", er.errno)
    print("Error Message:", er.msg)

107 Rows were successfully recorded in the Table Products.
CPU times: total: 31.2 ms
Wall time: 4.22 s


In [17]:
%%time

#Insert records into the Table Customers
try:
    insert_table_Customers_sql = """INSERT INTO little_lemon_org_database.Customers VALUES (%s,%s,%s)""" 
    for i,row in df_Customers.iterrows(): 
        cursor.execute(insert_table_Customers_sql, tuple(row)) 
        cnx.commit()
    print(i+1, "Rows were successfully recorded in the Table Customers.")
except Error as er:
    print("Something went wrong.")
    print("Error Code:", er.errno)
    print("Error Message:", er.msg)

1000 Rows were successfully recorded in the Table Customers.
CPU times: total: 766 ms
Wall time: 48 s


In [18]:
%%time

#Insert records into the Table Orders
try:
    insert_table_Orders_sql = """INSERT INTO little_lemon_org_database.Orders VALUES (%s,%s,%s)""" 
    for i,row in df_Orders.iterrows(): 
        cursor.execute(insert_table_Orders_sql, tuple(row)) 
        cnx.commit()
    print(i+1, "Rows were successfully recorded in the Table Orders.")
except Error as er:
    print("Something went wrong.")
    print("Error Code:", er.errno)
    print("Error Message:", er.msg)

1000 Rows were successfully recorded in the Table Orders.
CPU times: total: 594 ms
Wall time: 53.4 s


In [19]:
%%time

#Insert records into the Table OrderDetails
try:
    insert_table_OrderDetails_sql = """INSERT INTO little_lemon_org_database.OrderDetails(
                                            DeliveryDate, DeliveryCost, Quantity, CostPrice, 
                                            Discount, AddressID, OrderID, ProductID) 
                                            VALUES (%s,%s,%s,%s,%s,%s,%s,%s)""" 
    for i,row in df_OrderDetails.iterrows(): 
        cursor.execute(insert_table_OrderDetails_sql, tuple(row)) 
        cnx.commit()
    print(i+1, "Rows were successfully recorded in the Table OrderDetails.")
except Error as er:
    print("Something went wrong.")
    print("Error Code:", er.errno)
    print("Error Message:", er.msg)

21000 Rows were successfully recorded in the Table OrderDetails.
CPU times: total: 17.3 s
Wall time: 19min 30s


# Task 3: Create a stored procedure that returns the maximum quantity in an order. 

In [20]:
#Stored Procedure GetMaxQuantity() that returns the maximum quantity in all orders
DropGetMaxQuantity_sql ="""DROP PROCEDURE IF EXISTS GetMaxQuantity;"""
GetMaxQuantity_sql = """CREATE PROCEDURE IF NOT EXISTS GetMaxQuantity (
    OUT MaxQuantity INT)
    BEGIN 
        SELECT MAX(Quantity)  
        FROM `orderdetails`
        INTO MaxQuantity;
    END"""

#Create the Stored Procedure
try:
    cursor.execute(DropGetMaxQuantity_sql) 
    cursor.execute(GetMaxQuantity_sql) 
    print("Procedure successfully created or already exists.")
except Error as er:
    print("Something went wrong.")
    print("Error Code:", er.errno)
    print("Error Message:", er.msg) 

Procedure successfully created or already exists.


In [21]:
#Retrieve data from the Stored Procedure
args=['@MaxQuantity']

try:
    allResults=cursor.callproc('GetMaxQuantity', args)
    for eachResult in allResults:
        print('Maximum Quantity: ',eachResult)
except Error as er:
    print("Something went wrong.")
    print("Error Code:", er.errno)
    print("Error Message:", er.msg)  

Maximum Quantity:  3


In [22]:
#Stored Procedure GetMaxQuantity() that returns the maximum quantity in an order
DropGetAllMaxQuantity_sql ="""DROP PROCEDURE IF EXISTS GetAllMaxQuantity;"""
GetAllMaxQuantity_sql ="""CREATE PROCEDURE IF NOT EXISTS GetAllMaxQuantity ()
    BEGIN 
        SELECT OrderID, MAX(Quantity) AS 'Max Quantity' 
        FROM `orderdetails` 
        GROUP BY OrderID;
    END"""

#Create the Stored Procedure
try:
    cursor.execute(DropGetAllMaxQuantity_sql) 
    cursor.execute(GetAllMaxQuantity_sql) 
    print("Procedure successfully created or already exists.")
except Error as er:
    print("Something went wrong.")
    print("Error Code:", er.errno)
    print("Error Message:", er.msg) 

Procedure successfully created or already exists.


In [23]:
#Retrieve data from the Stored Procedure
try:
    cursor.callproc('GetAllMaxQuantity')
    allResults=[]
    for eachResult in cursor.stored_results():
        ds = eachResult.fetchall()
        i=1
        for eachResult in ds:
            print('Result #', i, "- OrderID: ",eachResult[0],"- Quantity: ",eachResult[1])
            i=i+1
except Error as er:
    print("Something went wrong.")
    print("Error Code:", er.errno)
    print("Error Message:", er.msg)  

Result # 1 - OrderID:  00-008-3271 - Quantity:  2
Result # 2 - OrderID:  00-283-3544 - Quantity:  2
Result # 3 - OrderID:  00-350-1396 - Quantity:  1
Result # 4 - OrderID:  00-451-8857 - Quantity:  1
Result # 5 - OrderID:  00-465-7031 - Quantity:  3
Result # 6 - OrderID:  00-618-9205 - Quantity:  3
Result # 7 - OrderID:  00-669-6040 - Quantity:  2
Result # 8 - OrderID:  00-799-3091 - Quantity:  3
Result # 9 - OrderID:  00-822-8333 - Quantity:  3
Result # 10 - OrderID:  00-965-9450 - Quantity:  2
Result # 11 - OrderID:  01-062-3384 - Quantity:  1
Result # 12 - OrderID:  01-423-8434 - Quantity:  3
Result # 13 - OrderID:  01-715-3869 - Quantity:  3
Result # 14 - OrderID:  01-733-5889 - Quantity:  1
Result # 15 - OrderID:  01-754-7255 - Quantity:  2
Result # 16 - OrderID:  01-855-8188 - Quantity:  3
Result # 17 - OrderID:  01-877-6020 - Quantity:  2
Result # 18 - OrderID:  01-879-8004 - Quantity:  3
Result # 19 - OrderID:  01-975-7149 - Quantity:  3
Result # 20 - OrderID:  01-981-5069 - Qu

# Task 4: Create a stored procedure that manages booking. 

In [48]:
# Create a Virtual Table (View) that returns all rows from all tables 
DropDataFromAllTables_sql = """DROP VIEW IF EXISTS DataFromAllTables;"""
DataFromAllTables_sql = """CREATE VIEW DataFromAllTables AS
SELECT da.AddressID, da.City, da.Country, da.PostalCode, da.CountryCode,
p.ProductID, p.CourseName, p.CuisineName, p.StarterName, p.DesertName, p.DrinkName, p.SidesName,
c.CustomerID, c.FirstName, c.LastName,
o.OrderID, o.OrderDate,
od.OrderdetailsID, od.DeliveryDate, od.DeliveryCost, od.Quantity, od.CostPrice, od.Salesprice,
od.Discount
FROM DeliveryAddress da, Products p, Customers c, Orders o, OrderDetails od 
WHERE c.CustomerID = o.CustomerID AND da.AddressID=od.AddressID AND
o.OrderID=od.OrderID AND p.ProductID=od.ProductID;"""

#Create the View
try:
    cursor.execute(DropDataFromAllTables_sql) 
    cursor.execute(DataFromAllTables_sql) 
    print("View successfully created or already exists.")
except Error as er:
    print("Something went wrong.")
    print("Error Code:", er.errno)
    print("Error Message:", er.msg) 

View successfully created or already exists.


In [25]:
#Stored Procedure ManageBooking() that selects data about the last 5 bookings
DropManageBooking_sql ="""DROP PROCEDURE IF EXISTS ManageBooking;"""
ManageBooking_sql = """CREATE PROCEDURE IF NOT EXISTS ManageBooking()
    BEGIN 
        SELECT DISTINCT (OrderID), CustomerID, OrderDate, AddressID, DeliveryDate  
        FROM `DataFromAllTables` 
        ORDER BY OrderDate DESC
        LIMIT 5;
    END;"""

#Create the Stored Procedure
try:
    cursor.execute(DropManageBooking_sql) 
    cursor.execute(ManageBooking_sql) 
    print("Procedure successfully created or already exists.")
except Error as er:
    print("Something went wrong.")
    print("Error Code:", er.errno)
    print("Error Message:", er.msg) 

Procedure successfully created or already exists.


In [49]:
#Retrieve data from the Stored Procedure
try:
    cursor.callproc('ManageBooking')
    allResults=[]
    for eachResult in cursor.stored_results():
        ds = eachResult.fetchall()
        i=1
        for eachResult in ds:
            print('Result #', i, "- OrderID: ",eachResult[0],"- CustomerID: ",eachResult[1],
                                 "- OrderDate: ",eachResult[2],"- AddressID: ",eachResult[3],"- DeliveryDate: ",eachResult[4])
            i=i+1
except Error as er:
    print("Something went wrong.")
    print("Error Code:", er.errno)
    print("Error Message:", er.msg)  

Result # 1 - OrderID:  57-000-2024 - CustomerID:  99-788-9999 - OrderDate:  2024-04-02 - AddressID:  MONTREAL-CAN-H1N4G6-1 - DeliveryDate:  2024-04-15
Result # 2 - OrderID:  52-708-9153 - CustomerID:  91-757-8892 - OrderDate:  2023-01-06 - AddressID:  KANSASCITY-UNI-6496418-1 - DeliveryDate:  2022-11-18
Result # 3 - OrderID:  52-708-9153 - CustomerID:  91-757-8892 - OrderDate:  2023-01-06 - AddressID:  KANSASCITY-UNI-353748-1 - DeliveryDate:  2022-11-18
Result # 4 - OrderID:  52-708-9153 - CustomerID:  91-757-8892 - OrderDate:  2023-01-06 - AddressID:  KANSASCITY-UNI-9930300-1 - DeliveryDate:  2022-11-18
Result # 5 - OrderID:  52-708-9153 - CustomerID:  91-757-8892 - OrderDate:  2023-01-06 - AddressID:  KANSASCITY-UNI-9870562-1 - DeliveryDate:  2022-11-18


# Task 5: Create a stored procedure that adds bookings.

In [27]:
#Stored Procedure AddBooking() that inserts a booking into the database. 
DropAddBooking_sql ="""DROP PROCEDURE IF EXISTS AddBooking;"""
AddBooking_sql ="""CREATE PROCEDURE IF NOT EXISTS AddBooking( 
                    IN addressid VARCHAR(45), 
                    IN city VARCHAR(45),
                    IN country VARCHAR(45),
                    IN postalcode VARCHAR(45),
                    IN countrycode VARCHAR(45),

                    IN productid VARCHAR(45),
                    
                    IN customerid VARCHAR(45),
                    IN firstname VARCHAR(45),
                    IN lastname VARCHAR(45),

                    IN orderid VARCHAR(45),
                    
                    IN deliverydate DATE,
                    IN deliverycost DECIMAL,
                    IN quantity INT,
                    IN costprice DECIMAL,
                    IN discount DECIMAL)
    BEGIN
        DECLARE coursename VARCHAR(45) DEFAULT 'NULL';
        DECLARE cuisinename VARCHAR(45) DEFAULT 'NULL';
        DECLARE startername VARCHAR(45) DEFAULT 'NULL';
        DECLARE desertname VARCHAR(45) DEFAULT 'NULL';
        DECLARE drinkname VARCHAR(45) DEFAULT 'NULL';
        DECLARE sidesname VARCHAR(45) DEFAULT 'NULL';

        SELECT Products.CourseName FROM Products WHERE Products.ProductID=productid INTO coursename;
        SELECT Products.CuisineName FROM Products WHERE Products.ProductID=productid INTO cuisinename;
        SELECT Products.StarterName FROM Products WHERE Products.ProductID=productid INTO startername;
        SELECT Products.DesertName FROM Products WHERE Products.ProductID=productid INTO desertname;
        SELECT Products.DrinkName FROM Products WHERE Products.ProductID=productid INTO drinkname;
        SELECT Products.SidesName FROM Products WHERE Products.ProductID=productid INTO sidesname;
        
        IF (coursename IS NOT NULL AND cuisinename IS NOT NULL AND startername IS NOT NULL
            AND desertname IS NOT NULL AND drinkname IS NOT NULL AND sidesname IS NOT NULL)
            THEN 
                INSERT INTO DeliveryAddress
                    VALUES (addressid, city, country, postalcode, countrycode);

                INSERT INTO Customers 
                    VALUES (customerid, firstname, lastname);

                INSERT INTO Orders 
                    VALUES (orderid, now(), customerid);

                INSERT INTO OrderDetails(DeliveryDate, DeliveryCost, Quantity, CostPrice, Discount, AddressID, OrderID, ProductID) 
                    VALUES (deliverydate, deliverycost, quantity, costprice, discount, addressid, orderid, productid); 
        END IF;
    END;"""

#Create the Stored Procedure
try:
    cursor.execute(DropAddBooking_sql) 
    cursor.execute(AddBooking_sql) 
    print("Procedure successfully created or already exists.")
except Error as er:
    print("Something went wrong.")
    print("Error Code:", er.errno)
    print("Error Message:", er.msg) 

Procedure successfully created or already exists.


In [28]:
#Insert records via the Stored Procedure
args=['MONTREAL-CAN-H1N4G6-1','Montreal','Canada','H1N4G6','CA','CarItaFalAffRomFoc1','99-788-9999','Jane','Doe','57-000-2024','2024-04-15', 11.50, 6, 230.50, 33.00] 

try:
    cursor.callproc('AddBooking', args)
    cnx.commit()
    print("A new booking was successfully added in the database")
except Error as er:
    print("Something went wrong.")
    print("Error Code:", er.errno)
    print("Error Message:", er.msg)  

A new booking was successfully added in the database


In [50]:
#Check that the booking added previously is successfully recorded
#Select all records from the View
select_view_DataFromAllTables_sql = """SELECT * FROM  DataFromAllTables
                                WHERE OrderId='57-000-2024';"""

#Execute queries
try:
    cursor.execute(select_view_DataFromAllTables_sql) 
    result=cursor.fetchall()
    print('\n'.join([', '.join(map(str,t)) for t in result]))
    print()
except Error as er:
    print("Something went wrong.")
    print("Error Code:", er.errno)
    print("Error Message:", er.msg)

MONTREAL-CAN-H1N4G6-1, Montreal, Canada, H1N4G6, CA, CarItaFalAffRomFoc1, Carbonara, Italian, Falafel, Affogato, Roma Red wine, Focaccia, 99-788-9999, Jane, Doe, 57-000-2024, 2024-04-02, 21001, 2024-04-15, 12.00, 6, 231.00, 346.50, 33.00



# Task 6: Create a stored procedure that updates bookings.

In [30]:
#Stored Procedure AddBooking() that updates a booking into the database 
DropUpdateBooking_sql ="""DROP PROCEDURE IF EXISTS UpdateBooking;"""
UpdateBooking_sql ="""CREATE PROCEDURE IF NOT EXISTS UpdateBooking( 
                    IN orderdetailsid INT,
                    IN deliverydate DATE,
                    IN deliverycost DECIMAL,
                    IN quantity INT,
                    IN costprice DECIMAL,
                    IN discount DECIMAL,
                    IN productid VARCHAR(45))
    BEGIN
        UPDATE OrderDetails SET 
        OrderDetails.DeliveryDate=deliverydate,
        OrderDetails.DeliveryCost=deliverycost,
        OrderDetails.Quantity=quantity,
        OrderDetails.Costprice=costprice,
        OrderDetails.Discount=discount,
        OrderDetails.ProductID=productid 
        WHERE OrderDetails.OrderDetailsID = orderdetailsid;
    END;"""

#Create the Stored Procedure
try:
    cursor.execute(DropUpdateBooking_sql) 
    cursor.execute(UpdateBooking_sql) 
    print("Procedure successfully created or already exists.")
except Error as er:
    print("Something went wrong.")
    print("Error Code:", er.errno)
    print("Error Message:", er.msg) 

Procedure successfully created or already exists.


In [53]:
#Check the booking to be modified before modification
#Select all records from the View
select_view_DataFromAllTables_sql = """SELECT * FROM  DataFromAllTables
                                WHERE OrderDetailsId='21001';"""

#Execute queries
try:
    cursor.execute(select_view_DataFromAllTables_sql) 
    result=cursor.fetchall()
    print('\n'.join([', '.join(map(str,t)) for t in result]))
    print()
except Error as er:
    print("Something went wrong.")
    print("Error Code:", er.errno)
    print("Error Message:", er.msg)


MONTREAL-CAN-H1N4G6-1, Montreal, Canada, H1N4G6, CA, CarItaFalAffRomFoc1, Carbonara, Italian, Falafel, Affogato, Roma Red wine, Focaccia, 99-788-9999, Jane, Doe, 57-000-2024, 2024-04-02, 21001, 2024-04-15, 12.00, 6, 231.00, 346.50, 33.00



In [57]:
#Modify records via the Stored Procedure
args=[21001,'2024-05-30', 200, 19, 401.00, 12,'BeaTurTomIceCorPot1'] 

try:
    cursor.callproc('UpdateBooking', args)
    cnx.commit()
    print("The booking was successfully updated in the database")
except Error as er:
    print("Something went wrong.")
    print("Error Code:", er.errno)
    print("Error Message:", er.msg)  

The booking was successfully updated in the database


In [58]:
#Check the booking modified after modification
#Select all records from the View
select_view_DataFromAllTables_sql = """SELECT * FROM  DataFromAllTables
                                WHERE OrderDetailsId='21001';"""

#Execute queries
try:
    cursor.execute(select_view_DataFromAllTables_sql) 
    result=cursor.fetchall()
    print('\n'.join([', '.join(map(str,t)) for t in result]))
    print()
except Error as er:
    print("Something went wrong.")
    print("Error Code:", er.errno)
    print("Error Message:", er.msg)


MONTREAL-CAN-H1N4G6-1, Montreal, Canada, H1N4G6, CA, BeaTurTomIceCorPot1, Bean soup, Turkish, Tomato bread, Ice cream, Corfu Red Wine, Potato salad, 99-788-9999, Jane, Doe, 57-000-2024, 2024-04-02, 21001, 2024-05-30, 200.00, 19, 401.00, 601.50, 12.00



# Task 7: Create a stored procedure that cancels bookings.

In [34]:
#Stored Procedure AddBooking() that deletes a booking into the database 
DropCancelBooking_sql ="""DROP PROCEDURE IF EXISTS CancelBooking;"""
CancelBooking_sql ="""CREATE PROCEDURE IF NOT EXISTS CancelBooking( 
                    IN orderid VARCHAR(45))
    BEGIN
        DELETE FROM Orders 
        WHERE `Orders`.`OrderID` = orderid;
    END;"""

#Create the Stored Procedure
try:
    cursor.execute(DropCancelBooking_sql) 
    cursor.execute(CancelBooking_sql) 
    print("Procedure successfully created or already exists.")
except Error as er:
    print("Something went wrong.")
    print("Error Code:", er.errno)
    print("Error Message:", er.msg) 

Procedure successfully created or already exists.


In [60]:
#Check the booking to be cancelled before delation
#Select all records from the View
select_view_DataFromAllTables_sql = """SELECT * FROM  DataFromAllTables
                                WHERE OrderId='00-008-3271';"""

#Execute queries
try:
    cursor.execute(select_view_DataFromAllTables_sql) 
    result=cursor.fetchall()
    print('\n'.join([', '.join(map(str,t)) for t in result]))
    print()
except Error as er:
    print("Something went wrong.")
    print("Error Code:", er.errno)
    print("Error Message:", er.msg)

ZŁOTNIKI-POL-6496345-1, Złotniki, Poland, 649-6345, PL, PizGreTomCheItaBru1, Pizza, Greek, Tomato bread, Cheesecake, Italian Coffee, Bruschetta, 50-245-9029, Pansie, Alldis, 00-008-3271, 2021-05-24, 333, 2022-12-29, 22.25, 2, 310.00, 465.00, 67.77
ZŁOTNIKI-POL-86030300-1, Złotniki, Poland, 86030300, PL, PizGreTomCheItaBru1, Pizza, Greek, Tomato bread, Cheesecake, Italian Coffee, Bruschetta, 50-245-9029, Pansie, Alldis, 00-008-3271, 2021-05-24, 1333, 2022-12-29, 22.25, 2, 91.84, 137.76, 94.76
ZŁOTNIKI-POL-9870489-1, Złotniki, Poland, 987-0489, PL, PizGreTomCheItaBru1, Pizza, Greek, Tomato bread, Cheesecake, Italian Coffee, Bruschetta, 50-245-9029, Pansie, Alldis, 00-008-3271, 2021-05-24, 2333, 2022-12-29, 22.25, 2, 320.00, 480.00, 91.41
ZŁOTNIKI-POL-9930227-1, Złotniki, Poland, 993-0227, PL, PizGreTomCheItaBru1, Pizza, Greek, Tomato bread, Cheesecake, Italian Coffee, Bruschetta, 50-245-9029, Pansie, Alldis, 00-008-3271, 2021-05-24, 3333, 2022-12-29, 22.25, 2, 93.13, 139.70, 24.00
ZŁOTNI

In [61]:
#Delete records via the Stored Procedure
args=['00-008-3271'] 

try:
    cursor.callproc('CancelBooking', args)
    cnx.commit()
    print("The booking was successfully cancelled in the database")
except Error as er:
    print("Something went wrong.")
    print("Error Code:", er.errno)
    print("Error Message:", er.msg)  

The booking was successfully cancelled in the database


In [62]:
#Check the booking cancelled after delation
#Select all records from the View
select_view_DataFromAllTables_sql = """SELECT * FROM  DataFromAllTables
                                WHERE OrderId='00-008-3271';"""

#Execute queries
try:
    cursor.execute(select_view_DataFromAllTables_sql) 
    result=cursor.fetchall()
    if(result):
        print('\n'.join([', '.join(map(str,t)) for t in result]))
        print()
    else:
        print("An empty result set id returned. This order doesn't exist.")
except Error as er:
    print("Something went wrong.")
    print("Error Code:", er.errno)
    print("Error Message:", er.msg)

An empty result set id returned. This order doesn't exist.


# Optional Task - Drop the Tables, View and Procedures. 

In [38]:
#SQL code to drop the Procedures
dropAllDBItems=[
    """DROP PROCEDURE IF EXISTS GetMaxQuantity;""", 
    """DROP PROCEDURE IF EXISTS GetAllMaxQuantity;""", 
    """DROP PROCEDURE IF EXISTS ManageBooking;""",
    """DROP PROCEDURE IF EXISTS AddBooking;""",
    """DROP PROCEDURE IF EXISTS UpdateBooking;""",
    """DROP PROCEDURE IF EXISTS CancelBooking;"""
]

"""
#Drop the Procedures
try:
    for dropEachDBitem in dropAllDBItems:
        cursor.execute(dropEachDBitem) 
        print(dropEachDBitem,"was executed successfully.")
except Error as er:
    print("Something went wrong.")
    print("Error Code:", er.errno)
    print("Error Message:", er.msg)
"""

'\n#Drop the Procedures\ntry:\n    for dropEachDBitem in dropAllDBItems:\n        cursor.execute(dropEachDBitem) \n        print(dropEachDBitem,"was executed successfully.")\nexcept Error as er:\n    print("Something went wrong.")\n    print("Error Code:", er.errno)\n    print("Error Message:", er.msg)\n'

In [39]:
#SQL code to drop the View and Tables
dropAllDBItems=[
    """DROP VIEW DataFromAllTables;""",
    """DROP TABLE OrderDetails;""", 
    """DROP TABLE Orders;""",
    """DROP TABLE Customers;""",
    """DROP TABLE DeliveryAddress;""",
    """DROP TABLE Products;"""
]
"""
#Drop the View and Tables
try:
    for dropEachDBitem in dropAllDBItems:
        cursor.execute(dropEachDBitem) 
        print(dropEachDBitem,"was executed successfully.")
except Error as er:
    print("Something went wrong.")
    print("Error Code:", er.errno)
    print("Error Message:", er.msg)
"""

'\n#Drop the View and Tables\ntry:\n    for dropEachDBitem in dropAllDBItems:\n        cursor.execute(dropEachDBitem) \n        print(dropEachDBitem,"was executed successfully.")\nexcept Error as er:\n    print("Something went wrong.")\n    print("Error Code:", er.errno)\n    print("Error Message:", er.msg)\n'

# Final Task - Close the cursor and connection. 

In [40]:
#Disconnect the cursor
cursor.close()
print("The cursor was successfully closed.")

The cursor was successfully closed.


In [41]:
#Disconnect from MySQL
cnx.close()
print ("Successfully disconnected from MySQL.")

Successfully disconnected from MySQL.
